# Import

In [1]:
import sf_quant as sf
from sf_quant.data.exposures import load_exposures_by_date
from sf_quant.data import load_assets_by_date
import polars as pl
import datetime as dt

/home/boobus/projects/factor_momentum/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-26 12:55:16,452	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


# Betas and Returns

In [2]:
start = dt.date(2020, 1, 7)
end =  dt.date(2024, 1, 7)

date_range = pl.date_range(
    start=dt.date(2020, 1, 7),
    end=dt.date(2024, 1, 7),
    interval="1d",
    eager=True  # makes it return a Series directly
)

In [3]:
return_columns = ['date','barrid','price','return']
df_raw = sf.data.load_assets(start=start,end=end,in_universe=True,columns=return_columns)
df_raw

date,barrid,price,return
date,str,f64,f64
2021-06-30,"""USA06Z1""",12.51,-0.0799
2021-07-01,"""USA06Z1""",12.92,3.2774
2021-07-02,"""USA06Z1""",12.4,-4.0248
2021-07-06,"""USA06Z1""",12.47,0.5645
2021-07-07,"""USA06Z1""",12.24,-1.8444
…,…,…,…
2023-12-29,"""USBPM41""",28.1,-4.9713
2024-01-02,"""USBPM41""",26.89,-4.306
2024-01-03,"""USBPM41""",27.57,2.5288


In [4]:
#16 types of style/risk factors from barra
style_factors = ['BETA','DIVYILD','EARNQLTY','EARNYILD','GROWTH',
                 'LEVERAGE','LTREVRSL','MGMTQLTY','MIDCAP','MOMENTUM',
                 'NETRET','PROFIT','PROSPECT','RESVOL','SIZE','VALUE']
style_factors = [f'USSLOWL_{fac}' for fac in style_factors]

In [5]:
beta_list = []
beta_chunks = []
for i, date in enumerate(date_range):

    beta_list.append(load_exposures_by_date(date))
    
    percent_done = i/len(date_range)
    if (i+1) % 50 == 0:
        print(f'{percent_done*100:.2f}% Complete')
        betas_chunk = (pl.concat(beta_list).select(['date', 'barrid'] + style_factors)
                       .join(df_raw, on=['date', 'barrid'], how='inner'))
        
        
        betas_chunk.write_csv(f'/home/boobus/temp/betas{int((i+1)/50)}.csv')
        beta_list = []

betas_chunk = (pl.concat(beta_list).select(['date', 'barrid'] + style_factors)
               .join(df_raw, on=['date', 'barrid'], how='inner'))
betas_chunk.write_csv(f'/home/boobus/temp/betas{int((i+1)/50)}.csv')
beta_list = []

3.35% Complete
6.77% Complete
10.19% Complete
13.61% Complete
17.03% Complete
20.45% Complete
23.87% Complete
27.29% Complete
30.71% Complete
34.13% Complete
37.55% Complete
40.97% Complete
44.39% Complete
47.81% Complete
51.23% Complete
54.65% Complete
58.07% Complete
61.49% Complete
64.91% Complete
68.33% Complete
71.75% Complete
75.17% Complete
78.59% Complete
82.01% Complete
85.43% Complete
88.85% Complete
92.27% Complete
95.69% Complete
99.11% Complete


In [6]:
chunks = []
for i in range(30):
    chunk = pl.read_csv(f'/home/boobus/temp/betas{i}.csv', infer_schema_length=10000, try_parse_dates=True)
    chunks.append(chunk)

df = pl.concat(chunks)

ShapeError: unable to append to a DataFrame of width 18 with a DataFrame of width 20

In [9]:
load_exposures_by_date(dt.date(2000, 1, 6)).select(['date', 'barrid'] + style_factors)

date,barrid,USSLOWL_BETA,USSLOWL_DIVYILD,USSLOWL_EARNQLTY,USSLOWL_EARNYILD,USSLOWL_GROWTH,USSLOWL_LEVERAGE,USSLOWL_LTREVRSL,USSLOWL_MGMTQLTY,USSLOWL_MIDCAP,USSLOWL_MOMENTUM,USSLOWL_NETRET,USSLOWL_PROFIT,USSLOWL_PROSPECT,USSLOWL_RESVOL,USSLOWL_SIZE,USSLOWL_VALUE
date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2000-01-06,"""AUSBYA1""",-2.582,1.172,0.34,1.6,-0.518,0.119,0.924,0.798,-0.329,-0.497,null,0.4,-0.902,0.438,0.641,0.163
2000-01-06,"""AUSDCQ1""",-0.172,-0.642,0.36,-0.039,0.851,-0.415,0.791,-0.656,0.369,-0.559,null,0.385,-0.219,-2.387,-2.311,-0.287
2000-01-06,"""BELAM51""",-1.126,-0.387,0.48,0.024,-0.32,0.122,-0.47,0.783,-0.361,-0.037,0.019,0.205,0.135,1.789,0.693,-0.277
2000-01-06,"""CANASK1""",-2.58,-0.642,-1.192,0.627,0.144,-0.415,2.516,-0.151,-2.79,-0.548,null,-0.782,-0.013,-2.814,-4.97,1.409
2000-01-06,"""CANAUD1""",-2.58,-0.642,0.329,2.104,-0.733,2.129,0.812,0.577,-0.352,-0.548,null,-0.374,-1.019,0.4,0.68,0.728
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2000-01-06,"""USAZZF1""",-1.141,-0.724,-2.225,-2.294,0.396,-0.721,1.396,-2.787,-2.79,-2.083,null,-0.472,2.152,2.743,-4.055,-0.507
2000-01-06,"""USAZZJ1""",-1.194,-0.642,-1.967,-0.176,-0.43,-0.415,1.89,-0.253,-2.79,-1.452,null,-0.387,0.288,-0.234,-4.97,1.214
2000-01-06,"""USAZZS1""",-2.585,-0.642,-0.129,-0.859,0.688,-0.555,0.138,-0.589,0.195,-1.383,null,-0.558,0.475,0.568,-2.439,1.234


In [10]:
load_exposures_by_date(dt.date(2000, 1, 7)).select(['date', 'barrid'] + style_factors)

date,barrid,USSLOWL_BETA,USSLOWL_DIVYILD,USSLOWL_EARNQLTY,USSLOWL_EARNYILD,USSLOWL_GROWTH,USSLOWL_LEVERAGE,USSLOWL_LTREVRSL,USSLOWL_MGMTQLTY,USSLOWL_MIDCAP,USSLOWL_MOMENTUM,USSLOWL_NETRET,USSLOWL_PROFIT,USSLOWL_PROSPECT,USSLOWL_RESVOL,USSLOWL_SIZE,USSLOWL_VALUE
date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2000-01-07,"""AUSBYA1""",-2.65,1.176,0.333,1.633,-0.526,0.134,0.898,0.8,-0.318,-0.509,null,0.393,-0.924,0.444,0.625,0.188
2000-01-07,"""AUSDCQ1""",0.071,-0.637,0.353,-0.033,0.844,-0.409,0.775,-0.655,0.349,-0.608,null,0.382,-0.233,-2.459,-2.327,-0.283
2000-01-07,"""BELAM51""",-1.469,-0.382,0.476,0.033,-0.328,0.13,-0.454,0.785,-0.36,0.082,0.019,0.198,0.073,2.015,0.694,-0.267
2000-01-07,"""CANASK1""",-2.647,-0.637,-1.19,0.625,0.128,-0.409,2.512,-0.141,-2.795,-0.561,null,-0.79,0.016,-2.806,-4.982,1.407
2000-01-07,"""CANAUD1""",-2.647,-0.637,0.323,2.115,-0.741,2.149,0.799,0.579,-0.358,-0.561,null,-0.381,-1.065,0.425,0.692,0.74
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2000-01-07,"""USAZZF1""",-1.347,-0.72,-2.223,-2.275,0.385,-0.714,1.389,-2.783,-2.795,-1.867,null,-0.48,2.138,2.722,-4.1,-0.502
2000-01-07,"""USAZZJ1""",-1.343,-0.637,-1.925,-0.168,-0.445,-0.409,1.92,-0.245,-2.795,-1.431,null,-0.398,0.265,-0.173,-4.982,1.195
2000-01-07,"""USAZZS1""",-2.39,-0.637,-0.146,-0.855,0.681,-0.548,0.135,-0.588,0.182,-1.388,null,-0.565,0.46,0.36,-2.45,1.263


In [11]:
sf.data.covariances.load_covariances_by_date(dt.date(2020,6,1))

date,factor_1,USSLOWL_AERODEF,USSLOWL_AIRLINES,USSLOWL_ALUMSTEL,USSLOWL_APPAREL,USSLOWL_AUTO,USSLOWL_BANKS,USSLOWL_BETA,USSLOWL_BEVTOB,USSLOWL_BIOLIFE,USSLOWL_BLDGPROD,USSLOWL_CHEM,USSLOWL_CNSTENG,USSLOWL_CNSTMACH,USSLOWL_CNSTMATL,USSLOWL_COMMEQP,USSLOWL_COMPELEC,USSLOWL_COMSVCS,USSLOWL_CONGLOM,USSLOWL_CONTAINR,USSLOWL_COUNTRY,USSLOWL_DISTRIB,USSLOWL_DIVFIN,USSLOWL_DIVYILD,USSLOWL_EARNQLTY,USSLOWL_EARNYILD,USSLOWL_ELECEQP,USSLOWL_ELECUTIL,USSLOWL_FOODPROD,USSLOWL_FOODRET,USSLOWL_GASUTIL,USSLOWL_GROWTH,USSLOWL_HLTHEQP,USSLOWL_HLTHSVCS,USSLOWL_HOMEBLDG,USSLOWL_HOUSEDUR,…,USSLOWL_LEVERAGE,USSLOWL_LIFEINS,USSLOWL_LIQUIDTY,USSLOWL_LTREVRSL,USSLOWL_MEDIA,USSLOWL_MGDHLTH,USSLOWL_MGMTQLTY,USSLOWL_MIDCAP,USSLOWL_MOMENTUM,USSLOWL_MULTUTIL,USSLOWL_NETRET,USSLOWL_OILGSCON,USSLOWL_OILGSDRL,USSLOWL_OILGSEQP,USSLOWL_OILGSEXP,USSLOWL_PAPER,USSLOWL_PHARMA,USSLOWL_PRECMTLS,USSLOWL_PROFIT,USSLOWL_PROSPECT,USSLOWL_PSNLPROD,USSLOWL_REALEST,USSLOWL_RESTAUR,USSLOWL_RESVOL,USSLOWL_ROADRAIL,USSLOWL_SEMICOND,USSLOWL_SEMIEQP,USSLOWL_SIZE,USSLOWL_SOFTWARE,USSLOWL_SPLTYRET,USSLOWL_SPTYCHEM,USSLOWL_SPTYSTOR,USSLOWL_TELECOM,USSLOWL_TRADECO,USSLOWL_TRANSPRT,USSLOWL_VALUE,USSLOWL_WIRELESS
date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2020-06-01,"""USSLOWL_AERODEF""",291.686909,168.765986,null,62.359255,-13.536614,-13.681855,null,-9.378598,-34.637063,46.379704,null,30.543742,17.453697,null,-18.386187,-15.0064,4.4959361,66.1962,null,101.212589,-19.135063,8.291803,null,null,null,34.273142,5.020318,-43.841724,-35.083343,-53.312658,null,0.87504,-9.175637,108.397138,26.424527,…,null,58.448541,null,null,-0.532614,15.503818,null,null,null,-29.515597,-18.146703,null,null,null,null,null,-23.879184,null,null,null,-20.125502,18.62498,49.417466,null,23.808474,-41.530097,-14.113695,null,-18.040191,10.251024,null,35.860829,-31.746701,31.780566,-19.016149,null,-79.198701
2020-06-01,"""USSLOWL_AIRLINES""",null,1066.21885,null,134.779816,78.432705,10.673013,null,8.79305,-58.58816,null,null,null,null,null,-28.113937,-3.005692,null,null,null,86.085836,5.358792,28.071227,null,null,null,null,20.403648,-59.870829,-57.386744,-59.553317,null,17.503486,21.983649,162.007926,83.181689,…,null,61.616133,null,null,2.929027,114.5629,null,null,null,-34.371172,-1.188113,null,null,null,null,null,-33.458149,null,null,null,-21.004848,70.209954,93.143089,null,48.482879,-39.147077,32.993895,null,-25.768959,73.925173,null,86.404651,-42.743259,null,34.060143,null,-135.908673
2020-06-01,"""USSLOWL_ALUMSTEL""",28.782102,-8.874692,380.242911,-18.164561,1.056769,11.789006,null,-12.408787,-18.058146,9.320994,null,77.812419,98.271152,null,-27.786206,3.218904,8.382864,85.036426,null,19.655236,17.80724,-0.494995,null,null,null,51.374276,-36.577343,-4.098606,-5.136807,-7.892131,null,-22.360653,-17.439657,-67.865633,-12.821851,…,null,-15.463502,null,null,-12.14988,-19.220501,null,null,null,-12.697624,-24.628699,null,null,null,null,null,-14.436756,162.16769,null,null,-10.562194,-20.69784,-38.643366,null,30.673874,-6.091401,-5.193226,null,-28.278184,-35.81512,null,-0.233976,0.404945,45.767428,41.385994,null,16.868624
2020-06-01,"""USSLOWL_APPAREL""",null,null,null,480.322148,null,-16.153442,null,0.295411,-59.850649,null,null,null,null,null,-32.017477,-10.557648,null,null,null,90.471031,null,8.572722,null,null,null,null,-35.028673,-57.729104,13.859437,-118.317426,null,-7.307761,2.061841,null,null,…,null,26.649662,null,null,null,-2.174935,null,null,null,-78.560401,null,null,null,null,null,null,-42.239286,null,null,null,-23.671859,41.631145,null,null,null,-52.06155,-28.090085,null,-37.173319,176.217698,null,220.648003,-49.924259,null,null,null,-63.130682
2020-06-01,"""USSLOWL_AUTO""",null,null,null,91.066647,322.933864,-12.434805,null,-14.93727,-45.368557,null,null,